Este dois exercícios são apresentados como introdução à modelação de problemas através de “SMT solvers” (Z3, CVC5,…) via uma das interfaces Python  (z3-solver  ou pySMT ) .

# Exercício 1

Neste exercício vai-se considerar a modelação de circuitos do 2º grau com falhas como estão descritos no documento +Trabalho prático: circuito com falhas . Recomenda-se a leitura atenta desse documento antes de considerar o resto do enunciado deste exercício.

Construa uma resolução das seguintes questões a partir de “inputs” do problema: os parâmetros $\,\kappa\,,\,n\,$ e de probabilidade de falha $\,\varepsilon\,$ restrita apenas às “gates” and.


1. Construa algoritmos para, sob “inputs” do segredo $\,z\in\{0,1\}^n\,$ e da “chave mestra” $\,s\in\{0,1\}^\kappa\,$, construa o circuito. Adicionalmente a partir deste circuito , construa o modelo SMT do circuito com falhas.
2. Usando o modelo acima, tente construir uma possível estimativa  para $z$ numa execução com falhas não nulas; isto é, encontrar  $\,z'\in \{0,1\}^n\,$  que é raíz de todos os polinómios que formam o circuito e uma situação de falhas não nulas em “gates” and  que conduz a essa estimativa.
3. Supondo que o circuito não introduz falhas em “gates” mas permite falhas nos “inputs”, conhecido  $\,z\in\{0,1\}^n\,$ pretende-se maximizar a probabilidade de falhas no “input” do circuito sem que o “output” $\,0^n\,$ seja alterado.

$$f(p;x)=o⊕⟨a⋅x⟩⊕(⟨b⋅x⟩×⟨c⋅x⟩)$$

In [26]:
from z3 import *
import numpy as np
from functools import reduce

Primeiro farei a funcao sem falhas para poder calcular o $z$ 

In [ ]:
# funçao f(p;x)

def f_true(p, x):
    o, a, b, c = p

    # produto interno mod 2
    def dot(u, v):
        return int(np.bitwise_xor.reduce(u & v))  # retorna 0 ou 1

    ax = dot(a, x)
    bx = dot(b, x)
    cx = dot(c, x)

    return o ^ ax ^ (bx & cx)






Aqui 

In [42]:
# funçoes de criaçao helpers

def zip_quads(V):
    O,A,B,C = V
    return [(O[i], A[i], B[i], C[i]) for i in range(len(O))]

def make_seed(k):
    bits =  np.random.randint(0, 2, size=k).tolist()
    return int("".join(str(b) for b in bits), 2)

def make_secret(n):
    return np.random.randint(0, 2, size=n).tolist()

def make_locals_s(seed,n):
    rng = np.random.default_rng(seed)
    seeds = rng.integers(0, 2**32, size=n, dtype=np.uint32)
    return seeds

def gen_vector(seeds,n,Z): # gera o meu vetor O
    A, B, C, O = [],[],[],[]
    for seed in range(len(seeds)):
        rng = np.random.default_rng(int(seeds[seed]))
        a = rng.integers(0, 2, size=n, dtype=np.uint8)
        b = rng.integers(0, 2, size=n, dtype=np.uint8)
        c = rng.integers(0, 2, size=n, dtype=np.uint8)

        o = f_true((0,a,b,c), Z)

        A.append(a)
        B.append(b)
        C.append(c)
        O.append(o)

        
        
    #print("O ",O)
    #print("A ",A)
    #print("B ",B)
    #print("C ",C)
    return (O,A,B,C)


In [75]:

k = 16

Z = [1,1,0,0,0,0,0,0,0,1,1]
print(Z)
n = len(Z)

#seed = make_seed(k)
seed = 69
#print(seed)

seeds = make_locals_s(seed,n)
#print(seeds)



vecs = gen_vector(seeds,n,Z)
#print(vecs)

O = vecs[0]

p = zip_quads(vecs)
#print(p)



#TERMINADO O SETUP!!! agora basta fazer o solver

[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [76]:
q = (O,seed) # com esse q preciso achar o meu z!
print(q)

([1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1], 69)


In [77]:
def regen_params(seed, n):
    A, B, C = [], [], []
    seeds = make_locals_s(seed,n)
    for seed in range(len(seeds)):
        rng = np.random.default_rng(int(seeds[seed]))  # mesma inicialização
        a = rng.integers(0, 2, size=n, dtype=np.uint8)
        b = rng.integers(0, 2, size=n, dtype=np.uint8)
        c = rng.integers(0, 2, size=n, dtype=np.uint8)
        A.append(a)
        B.append(b)
        C.append(c)
    return A, B, C

def dot(a, z):
    return reduce(lambda x, y: Xor(x, y),
                  [z[j] for j in range(len(a)) if a[j]],
                  BoolVal(False))



In [78]:
def decrypt_theoric(q, n):
    O, seed = q
    A, B, C = regen_params(seed, n)

    #print("O ", O)
    #print("A ", A)
    #print("B ", B)
    #print("C ", C)

    Z = [Bool(f"z_{i}") for i in range(n)]
    
    s = Solver()

    for i in range(n):
        a, b, c, o = A[i], B[i], C[i], O[i]




        offset = BoolVal(o == 1)
        dots = Xor(dot(a, Z), And(dot(b, Z), dot(c, Z)))  # isto devia ter uma chance de erro

        s.add(Not(Xor(offset, dots)))

    if s.check() == sat:
        model = s.model()
        z_val = [model[zj] for zj in Z]
        print(''.join(['1' if model[zj] else '0' for zj in Z]))
        return [bool(model[zj]) for zj in Z]
    else:
        print("nenhuma soluçao encontrada")
        return None


decrypt_theoric(q, n)


    

    

10111010101


[True, False, True, True, True, False, True, False, True, False, True]

No caso das SMT, onde é possível usar restrições disjuntivas sem grandes custos, é conveniente modelar cada “wire” $\,i\,$ do circuito por um par de variáveis booleanas $\,d_i\,,\,w_i\,$. A primeira variável $\,d_i\,$ determina se o “wire” tem um valor bem definido ou tem uma falha; no caso de ter um valor bem definido esse valor está apresentado por $\,w_i\,$.

In [79]:
def decrypt_real(q, n,red): #now adding redundancy
    O, seed = q
    A, B, C = regen_params(seed, n)

    #print("O ", O)
   # print("A ", A)
    #print("B ", B)
    #print("C ", C)

    Z = [Bool(f"z_{i}") for i in range(n)]
    E = [(Int(f"d_{i}")) for i in range(red*n)] # 0: nao existe falha; 1: existe falha

    d_vars = [d for d in E]  # pega só os d_i
    
    s = Optimize()

    for d in E:
        s.add(Or(d == 0, d == 1))

    s.add(Sum(E) > 0)

    contas_valids = []
    
    for i in range(n):
        
        a, b, c, o = A[i], B[i], C[i], O[i]

        offset = BoolVal(o == 1)
        
        red_v = []
        and_v = If(And(dot(b, Z), dot(c, Z)),1,0)
        
        for j in range(red):
            d_j = E[i*red + j]
            v = If(d_j == 0, and_v,IntVal(-1) )
            red_v.append(v)
            

        count_valid = Sum([If(Or(v == 0, v == 1), 1, 0) for v in red_v]) #isso eh o majority
        contas_valids.append(count_valid)
        
        s.add(count_valid > Sum([If(v == -1, 1, 0) for v in red_v]))
        

        dots = Xor(dot(a, Z), And(dot(b, Z), dot(c, Z)))  # isto devia ter uma chance de erro

        s.add(Not(Xor(offset, dots)))
        

    total = Sum(contas_valids)
    
    
    s.minimize(total)

    if s.check() == sat:
        model = s.model()
        z_val = [is_true(model[zj]) for zj in Z]
        print("Z =", ''.join(['1' if val else '0' for val in z_val]))
    
        wires_on = []
        wires_fail = []
    
        for i, (d) in enumerate(E):
            d_val = model.evaluate(d, model_completion=True).as_long()  
    
            if d_val == 1:   
                wires_fail.append(i)
            else:            
                wires_on.append(i)
    
        total_wires = len(E)
        fail_ratio = len(wires_fail) / total_wires * 100
    
        print(f"\n=== Status dos wires ===")
        print(f"Funcionando ({len(wires_on)}): {wires_on}")
        print(f"Falhando ({len(wires_fail)}): {wires_fail}")
        print(f"Probabiliade de falha: {fail_ratio:.2f}%\n")
    
        return z_val
    else:
        print("Nenhuma solução encontrada")
        return None


decrypt_real(q, n,9)

Z = 10111010101

=== Status dos wires ===
Funcionando (55): [0, 1, 2, 4, 6, 9, 10, 14, 15, 17, 20, 23, 24, 25, 26, 27, 30, 31, 34, 35, 36, 40, 41, 43, 44, 46, 47, 49, 52, 53, 56, 57, 58, 61, 62, 63, 64, 65, 67, 68, 72, 75, 76, 77, 78, 81, 83, 86, 87, 88, 91, 92, 93, 95, 98]
Falhando (44): [3, 5, 7, 8, 11, 12, 13, 16, 18, 19, 21, 22, 28, 29, 32, 33, 37, 38, 39, 42, 45, 48, 50, 51, 54, 55, 59, 60, 66, 69, 70, 71, 73, 74, 79, 80, 82, 84, 85, 89, 90, 94, 96, 97]
Probabiliade de falha: 44.44%



[True, False, True, True, True, False, True, False, True, False, True]